## Configuración del enviroment

In [2]:
%pip install opencv_python

In [3]:
%pip install matplotlib

In [4]:
%pip install ultralytics

# Importaciones necesarias

In [1]:
import math
import cv2
import numpy as np

from ultralytics import YOLO

In [2]:
# Carga del modelo
model = YOLO('yolov8n.pt')

# Calculo del punto de detención

Función auxiliar para determinar la linea de stop y el punto de detención donde se deberá completar la parada para validar el stop del vehiculo

In [3]:
def set_stop_point(frame):

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img_suavizada = cv2.GaussianBlur(img, (3, 3), 0)
    
    ## Aplicación de sobel
    # Calcula en ambas direcciones (horizontal y vertical)
    sobelx = cv2.Sobel(img_suavizada, cv2.CV_64F, 1, 0)  # x
    sobely = cv2.Sobel(img_suavizada, cv2.CV_64F, 0, 1)  # y
    # Combina ambos resultados
    sobel = cv2.add(sobelx, sobely)
    sobel8 = np.uint8(sobel)
    # Calcula en ambas direcciones (horizontal y vertical)
    sobel8x = np.uint8(sobelx)
    sobel8y = np.uint8(sobely)

    ## Calculos de máximos
    # Calcula el valor máximo en columnas
    max_cols = max([sum(columna) for columna in sobel8y])
    # Calcula el valor máximo en filas
    max_rows = max([sum(fila) for fila in sobel8x])

    index_cols = [index for index,columna in enumerate(sobel8y) if sum(columna) > 0.95*max_cols]
    index_rows = [index for index,fila in enumerate(sobel8x) if sum(fila) > 0.95*max_rows]

    print("Punto de parada:",max(index_cols),max(index_rows))
    return (max(index_cols),max(index_rows))

# Analisis del movimiento del vehiculo

Ejemplo realizando el stop correctamente

In [12]:

vid = cv2.VideoCapture("correcto.mp4") #Busca n vídeo propio

stop_point = None
stoped = False
last_point = (0,0)
car_point = (0,0)
frame_counter = 0
frames_stoped_counter = 0
frames_stoped_goal = 50
frame_y_history = []

while(True):
    # fotograma a fotograma
    ret, frame = vid.read()

    if ret:

        if stop_point == None:
            stop_point = set_stop_point(frame)

        cv2.circle(frame,stop_point, 8, (255,0,255), -1)
        cv2.putText(frame, "Punto de parada", (stop_point[0], stop_point[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 2)

        # Perform inference on an image
        results = model(frame, stream=True)

        frame_counter += 1

        # Para cada detección
        for r in results:
            boxes = r.boxes
            for box in boxes:
                if int(box.cls[0]) == 2:
                    # Contenedor
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                    # Dibuja el contenedor del coche
                    if stoped:
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    else:
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)

                    # Dibuja punto intermedio coche
                    cv2.circle(frame,(x1+75,y2), 8, (255,0,255), -1)

                    # Guardamos Historico de las posiciones del coche
                    frame_y_history.append(y2)

                    # Contamos el tiempo que pasa el vehiculo parado
                    if frame_counter > 60:
                        if y2 - frame_y_history[-4] <= 1:
                            frames_stoped_counter += 1

                    # Si el vehiculo para suficiente tiempo y a la distancia adecuada de la linea verificamos el stop
                    if y2 >=  stop_point[1] and frames_stoped_counter >= frames_stoped_goal:
                        stoped = True       
                    
                

        if stoped:
            cv2.putText(frame, "Stop completado :)", (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        else:
            cv2.putText(frame, "Stop no completado :(", (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

        # Muestra fotograma
        cv2.imshow("correcto",frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(10) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Punto de parada: 112 451

0: 640x480 2 motorcycles, 127.1ms
Speed: 2.0ms preprocess, 127.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 motorcycles, 125.1ms
Speed: 3.0ms preprocess, 125.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 motorcycles, 114.1ms
Speed: 2.0ms preprocess, 114.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 motorcycles, 115.1ms
Speed: 3.0ms preprocess, 115.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 motorcycles, 120.1ms
Speed: 3.0ms preprocess, 120.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 bicycle, 2 motorcycles, 114.1ms
Speed: 2.0ms preprocess, 114.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 motorcycles, 119.1ms
Speed: 2.0ms preprocess, 119.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 motorcycles, 115.1m

Ejemplo realizando el stop incorrectamente

In [11]:
vid = cv2.VideoCapture("incorrecto.mp4") #Busca n vídeo propio

stop_point = None
stoped = False
last_point = (0,0)
car_point = (0,0)
frame_counter = 0
frames_stoped_counter = 0
frames_stoped_goal = 50
frame_y_history = []

while(True):
    # fotograma a fotograma
    ret, frame = vid.read()

    if ret:

        if stop_point == None:
            stop_point = set_stop_point(frame)

        cv2.circle(frame,stop_point, 8, (255,0,255), -1)
        cv2.putText(frame, "Punto de parada", (stop_point[0], stop_point[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,255), 2)

        # Perform inference on an image
        results = model(frame, stream=True)

        frame_counter += 1

        # Para cada detección
        for r in results:
            boxes = r.boxes
            for box in boxes:
                if int(box.cls[0]) == 2:
                    # Contenedor
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                    # Dibuja el contenedor del coche
                    if stoped:
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    else:
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)

                    # Dibuja punto intermedio coche
                    cv2.circle(frame,(x1+75,y2), 8, (255,0,255), -1)

                    frame_y_history.append(y2)

                    if frame_counter > 60:
                        if y2 - frame_y_history[-4] <= 1:
                            frames_stoped_counter += 1

                    if y2 >=  stop_point[1] and frames_stoped_counter >= frames_stoped_goal:
                        stoped = True       
                    
                

        if stoped:
            cv2.putText(frame, "Stop completado :)", (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        else:
            cv2.putText(frame, "Stop no completado :(", (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

        # Muestra fotograma
        cv2.imshow("incorrecto",frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(10) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Punto de parada: 79 477

0: 640x448 1 bicycle, 2 motorcycles, 125.1ms
Speed: 3.0ms preprocess, 125.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 1 bicycle, 2 motorcycles, 137.1ms
Speed: 3.0ms preprocess, 137.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 1 bicycle, 2 motorcycles, 116.1ms
Speed: 3.0ms preprocess, 116.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 1 bicycle, 2 motorcycles, 116.1ms
Speed: 2.0ms preprocess, 116.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 1 bicycle, 2 motorcycles, 181.2ms
Speed: 2.0ms preprocess, 181.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 1 bicycle, 2 motorcycles, 109.1ms
Speed: 3.0ms preprocess, 109.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x448 1 bicycle, 2 motorcycles, 107.1ms
Speed: 2.0ms preprocess, 107.1ms inference, 1.0ms postprocess per 